# Joins

In [32]:
def getTCNIngreso(l):
    e = l.split(",")
    tcn = e[0]
    ingreso = float(e[6])
    return (tcn, ingreso)

In [33]:
sc.textFile('data/coupon150720.csv').first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [34]:
cs = sc.textFile('data/coupon150720.csv').map(getTCNIngreso)

In [35]:
cs.take(5)

[(u'79062005698500', 56.79),
 (u'79062005698500', 84.34),
 (u'79062005924069', 60.0),
 (u'79065668570385', 160.63),
 (u'79065668737021', 152.46)]

In [36]:
def getTCNCanal(l):
    e = l.split(',')
    tcn = e[0]
    canal = e[8]
    return (tcn, canal)

In [37]:
ts = sc.textFile('data/transm150720.csv').map(getTCNCanal)

In [38]:
ts.take(2)

[(u'79062005698500', u'T'), (u'79062005924069', u'T')]

In [39]:
#cupones
cs.count()

1232662

In [40]:
#tickets
ts.count()

631405

### join 

In [41]:
# el join es igual que en base de datos pero la clave no
ts.leftOuterJoin(cs).take(3)

[(u'79062005994272', (u'A', 56.27)),
 (u'79062005994272', (u'A', 56.27)),
 (u'79065668614400', (u'T', 162.33))]

In [42]:
ts.leftOuterJoin(cs).count()

1232662

In [43]:
cs.leftOuterJoin(ts).count()

1232662

In [44]:
# Esto nos dice en cs hay claves repetidas

In [45]:
#Al hacerel join antes del reduce nos aseguramos las claves
j = ts.leftOuterJoin(cs)

In [46]:
j.take(2)

[(u'79062005994272', (u'A', 56.27)), (u'79062005994272', (u'A', 56.27))]

In [47]:
j.map(lambda x: x[1]).reduceByKey(lambda x,y: x+y).take(2)

[(u'', 345827.3100000002), (u'T', 83637221.2200021)]

In [48]:
j.map(lambda x: x[1]).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], False).take(10)

[(u'A', 99011336.9600026),
 (u'T', 83637221.2200021),
 (u'E', 994305.880000001),
 (u'V', 843207.13),
 (u'', 345827.3100000002)]

In [49]:
#valor de cada ticket agrupado por cupón
valorTs = cs.groupByKey()

In [50]:
valorTs.take(2)

[(u'79062005558463',
  <pyspark.resultiterable.ResultIterable at 0x7f1c42619d10>),
 (u'79065668432713',
  <pyspark.resultiterable.ResultIterable at 0x7f1c426191d0>)]

In [51]:
#cada lista devuelta trendrá como mucho 4 elementos (numero de tickets)
valorTs.mapValues(lambda vs: sum(vs)).take(2)

[(u'79062005558463', 0.0), (u'79065668432713', 286.98)]

In [54]:
valorTs = cs.groupByKey().mapValues(lambda vs: sum(vs))

In [55]:
valorTs.count()

631405

In [56]:
#las dos tablas tiene ahora el mismo tamaña
j = ts.leftOuterJoin(valorTs)

In [57]:
j.take(3)

[(u'79062005994272', (u'A', 112.54)),
 (u'79065668614400', (u'T', 229.51000000000002)),
 (u'79062005879562', (u'T', 99.50999999999999))]

In [59]:
j.map(lambda x: x[1]).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], False).take(5)

[(u'A', 99011336.96000284),
 (u'T', 83637221.22000207),
 (u'E', 994305.8800000008),
 (u'V', 843207.1299999999),
 (u'', 345827.31)]

####  Niños por aerolínea

In [60]:
#niños
ts = sc.textFile('data/transm150720.csv').map(lambda x: x.split(',')[13]).filter(lambda x: x=='CHD').count()

In [61]:
ts

40016

In [62]:
sc.textFile('data/transm150720.csv').map(lambda x: x.split(',')[13]).filter(lambda x: x=='CHD').take(3)

[u'CHD', u'CHD', u'CHD']

In [63]:
def getChildren(l):
    e = l.split(",")
    ticket = e[0]
    children = e[13]
    return(ticket, children)

In [64]:
sc.textFile('data/transm150720.csv').map(getChildren).take(3)

[(u'79062005698500', u'   '),
 (u'79062005924069', u'ADT'),
 (u'79065668570385', u'CHD')]

In [66]:
sc.textFile('data/transm150720.csv').map(getChildren).filter(lambda x: x[1]=='CHD').take(3)

[(u'79065668570385', u'CHD'),
 (u'79062005305018', u'CHD'),
 (u'79062006060658', u'CHD')]

In [67]:
cs = sc.textFile('data/coupon150720.csv').take(3)

In [68]:
cs

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0']

In [94]:
# SOL
def getTCNAerolinea(l):
    e = l.split(",")
    tcn = e[0]
    al = e[4]
    return (tcn, al)

In [95]:
als = sc.textFile('data/coupon150720.csv').map(getTCNAerolinea)

In [96]:
als.take(2)

[(u'79062005698500', u'9W'), (u'79062005698500', u'9W')]

In [97]:
fpax = sc.textFile('data/transm150720.csv').map(getChildren).filter(lambda x: x[1]=='CHD')

In [98]:
fpax.leftOuterJoin(als).take(2)

[(u'79065668446486', (u'CHD', u'UL')), (u'79065668446486', (u'CHD', u'AI'))]

In [99]:
fpax.leftOuterJoin(als).count()

73497

In [100]:
als.leftOuterJoin(fpax).count()

1232662

In [104]:
#vemos que la primera forma es más óptima
als.rightOuterJoin(fpax).count()


73497

In [105]:
#por lo tanto
als.rightOuterJoin(fpax).take(2)

[(u'79062006033668', (u'LH', u'CHD')), (u'79062006033668', (u'OS', u'CHD'))]

In [106]:
j = als.rightOuterJoin(fpax).map(lambda x: x[1])

In [107]:
j.take(2)

[(u'LH', u'CHD'), (u'OS', u'CHD')]

In [108]:
#sólo los niños
j.mapValues(lambda x: 1).take(2)

[(u'LH', 1), (u'OS', 1)]

In [109]:
j.mapValues(lambda x: 1).reduceByKey(lambda x,y: x+y ).take(2)

[(u'', 969), (u'FR', 1)]

In [110]:
j.mapValues(lambda x: 1).reduceByKey(lambda x,y: x+y ).sortBy(lambda x: x[1], False).take(5)

[(u'JJ', 5324), (u'SV', 5072), (u'BA', 3417), (u'AB', 3214), (u'AF', 3118)]

### Los 5 cupones más caros de niños

In [111]:
def getChildrenImporte(l):
    e = l.split(",")
    ticket = e[0]
    importe = float(e[6])
    return(ticket, (children, importe))

In [115]:
sc.textFile('data/coupon150720.csv').map(getChildrenImporte).take(3)

[(u'79062005698500', (u'OK', 56.79)),
 (u'79062005698500', (u'OK', 84.34)),
 (u'79062005924069', (u'OK', 60.0))]

In [116]:
cupones = sc.textFile('data/coupon150720.csv').map(getChildrenImporte)

In [125]:
children = sc.textFile('data/transm150720.csv').map(getChildren).filter(lambda x: x[1]=='CHD')

In [126]:
children.take(2)

[(u'79065668570385', u'CHD'), (u'79062005305018', u'CHD')]

In [132]:
#sol

In [133]:
children.take(2)

[(u'79065668570385', u'CHD'), (u'79062005305018', u'CHD')]

In [134]:
cs = sc.textFile("data/coupon150720.csv").map(getTCNIngreso)

In [135]:
cs.take(2)

[(u'79062005698500', 56.79), (u'79062005698500', 84.34)]

In [136]:
children.count()

40016

In [138]:
children.join(cs).take(2)

[(u'79065668446486', (u'CHD', 165.69)), (u'79065668446486', (u'CHD', 172.37))]

In [140]:
children.join(cs).map(lambda x: x[1]).sortBy(lambda x: x[1], False).take(5)

[(u'CHD', 14498.76),
 (u'CHD', 13547.31),
 (u'CHD', 9446.75),
 (u'CHD', 7709.74),
 (u'CHD', 6629.51)]

#### cogroup

In [ ]:
# Calcular la fecha mínima de cada cupón y calculamos la diferencia de la mínima con la fecha de compra y vemos la antelación de la compra

In [ ]:
#Vamos a contar cupones por tiket primero

In [143]:
tcnCs = sc.textFile('data/coupon150720.csv').map(lambda x: x.split(',')[0])

In [144]:
tcnCs.take(2)

[u'79062005698500', u'79062005698500']

In [154]:
cupones = tcnCs.map(lambda x: (x,1))

In [155]:
cupones.take(2)

[(u'79062005698500', 1), (u'79062005698500', 1)]

In [147]:
tcnTs = sc.textFile('data/transm150720.csv').map(lambda x: x.split(',')[0])

In [156]:
tikets = tcnTs.map(lambda x: (x,1))

In [157]:
tikets.take(3)

[(u'79062005698500', 1), (u'79062005924069', 1), (u'79065668570385', 1)]

In [158]:
tikets.cogroup(cupones).take(2)

[(u'79062005994272',
  (<pyspark.resultiterable.ResultIterable at 0x7f1c42593310>,
   <pyspark.resultiterable.ResultIterable at 0x7f1c42593ad0>)),
 (u'79065668614400',
  (<pyspark.resultiterable.ResultIterable at 0x7f1c425931d0>,
   <pyspark.resultiterable.ResultIterable at 0x7f1c42593a90>))]

In [162]:
j = tikets.cogroup(cupones)

In [165]:
j.mapValues(lambda x: (len(x[0]), len(x[1]))).take(3)

[(u'79062005994272', (1, 2)),
 (u'79065668614400', (1, 2)),
 (u'79062005879562', (1, 2))]

In [ ]:
#comprobamos que todos son 1

In [166]:
j.mapValues(lambda x: (len(x[0]), len(x[1]))).map(lambda x: x[1][0]).distinct().take(5)

[1]

In [ ]:
# me sale 1, por lo que es correcto: no hay claves duplicadas. Todos los ticket en ficher tickets tienen clave única

In [167]:
j.mapValues(lambda x: (len(x[0]), len(x[1]))).sortBy(lambda x: x[1][1], False).take(5)

[(u'79065669300201', (1, 16)),
 (u'79062005712937', (1, 15)),
 (u'79062005779716', (1, 13)),
 (u'79065668940838', (1, 12)),
 (u'79062005319376', (1, 12))]

In [168]:
# más sencillo
j.mapValues(lambda x: len(x[1])).sortBy(lambda x: [1], False).take(5)

[(u'79062005994272', 2),
 (u'79065668614400', 2),
 (u'79062005879562', 2),
 (u'79062005474691', 3),
 (u'79062006175895', 4)]